# Robot Demo

Uses Matamoe repo layout:
- `common/lib`
- `common/sounds`
- `lessons/...`

Demos:
- TTS
- Eyes / RGB
- Camera
- Robot movement (best-effort)


In [ ]:
from pathlib import Path
import os, sys, time

# --- Find repo root (the folder that contains 'common' and 'lessons') ---
def find_repo_root(start: Path) -> Path:
    p = start.resolve()
    for _ in range(10):
        if (p / 'common').is_dir() and (p / 'lessons').is_dir():
            return p
        p = p.parent
    raise FileNotFoundError('Could not find repo root containing common/ and lessons/')

repo_root = find_repo_root(Path.cwd())
common_lib = repo_root / 'common' / 'lib'

# Add paths so imports work regardless of notebook location
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))
if str(common_lib) not in sys.path:
    sys.path.insert(0, str(common_lib))

print('Repo root:', repo_root)
print('common/lib:', common_lib)
print('sys.path[0:3]:', sys.path[0:3])

# --- Fix broken CycloneDDS config coming from shell ---
if os.environ.get('CYCLONEDDS_URI', '').strip() == 'file:///etc/cyclonedds/config.xml':
    print('Unsetting bad CYCLONEDDS_URI (missing /etc/cyclonedds/config.xml)')
    os.environ.pop('CYCLONEDDS_URI', None)

# Reasonable defaults if not set by supervisor
os.environ.setdefault('RMW_IMPLEMENTATION', 'rmw_cyclonedds_cpp')
os.environ.setdefault('ROS_DOMAIN_ID', '0')

print('ROS_DOMAIN_ID =', os.environ.get('ROS_DOMAIN_ID'))
print('RMW_IMPLEMENTATION =', os.environ.get('RMW_IMPLEMENTATION'))
print('CYCLONEDDS_URI =', os.environ.get('CYCLONEDDS_URI'))
print('Ready')

## Imports

Imports from `common/lib` directly (no `common` package needed).

In [ ]:
import rclpy

import robot as robot_lib
import camera_lib
import tts_lib
import eyes_lib

print('Imports OK')
print('robot_lib:', getattr(robot_lib, '__file__', '?'))
print('camera_lib:', getattr(camera_lib, '__file__', '?'))
print('tts_lib:', getattr(tts_lib, '__file__', '?'))
print('eyes_lib:', getattr(eyes_lib, '__file__', '?'))

## TTS

Uses your `tts_lib` helpers.

In [ ]:
def speak(text: str):
    if hasattr(tts_lib, 'say'):
        return tts_lib.say(text)
    if hasattr(tts_lib, 'speak'):
        return tts_lib.speak(text)
    if hasattr(tts_lib, 'tts_to_wav'):
        # fall back: generate wav then play if your sound helper exists
        wav = tts_lib.tts_to_wav(text)
        print('Generated wav:', wav)
        return wav
    raise RuntimeError('No say()/speak()/tts_to_wav() found in tts_lib')

try:
    speak('Kia ora team. Robot demo starting.')
    print('TTS OK')
except Exception as e:
    print('TTS failed:', e)

## Eyes / RGB

Uses your `eyes_lib` abstraction.
If this does nothing, check:
- `sonar_controller` running
- `smbus2` installed
- `ROS_DOMAIN_ID` matches the running ROS stack

In [ ]:
try:
    eyes = eyes_lib.get_eyes()
    print('Eyes object:', eyes)

    # If your eyes_lib supports diagnose/scan, use them
    if hasattr(eyes, 'diagnose'):
        print('diagnose():', eyes.diagnose())
    if hasattr(eyes, 'scan_indices'):
        print('scan_indices():', eyes.scan_indices())

    # Common patterns: set_rgb / set / colour / blink
    if hasattr(eyes, 'set_rgb'):
        eyes.set_rgb(0, 255, 0, 0); time.sleep(0.4)
        eyes.set_rgb(0, 0, 255, 0); time.sleep(0.4)
        eyes.set_rgb(0, 0, 0, 255); time.sleep(0.4)
        eyes.set_rgb(0, 0, 0, 0)
    elif hasattr(eyes, 'blink'):
        eyes.blink()
    else:
        print('Eyes API:', [n for n in dir(eyes) if not n.startswith('_')])

    print('Eyes OK')
except Exception as e:
    print('Eyes failed:', e)

## Camera

Best-effort snapshot.
Saves to `/tmp/robot_demo/snapshot.jpg` if possible.

In [ ]:
out_dir = Path('/tmp/robot_demo')
out_dir.mkdir(parents=True, exist_ok=True)
out_path = out_dir / 'snapshot.jpg'

try:
    if hasattr(camera_lib, 'snapshot'):
        camera_lib.snapshot(str(out_path))
        print('Saved', out_path)
    elif hasattr(camera_lib, 'get_frame'):
        frame = camera_lib.get_frame()
        try:
            import cv2
            cv2.imwrite(str(out_path), frame)
            print('Saved', out_path)
        except Exception as e:
            print('Got frame but could not save (cv2 missing?):', e)
    else:
        print('camera_lib API:', [n for n in dir(camera_lib) if not n.startswith('_')])
except Exception as e:
    print('Camera failed:', e)

## Movement

This prints the available movement API and tries a few common calls.
If it fails, you’ll see the available function names.

In [ ]:
def public_names(mod):
    return [n for n in dir(mod) if not n.startswith('_')]

print('robot_lib API:', public_names(robot_lib))

def try_call(obj, names, *args, **kwargs):
    for n in names:
        if hasattr(obj, n):
            fn = getattr(obj, n)
            return n, fn(*args, **kwargs)
    raise AttributeError('None exist: ' + ', '.join(names))

try:
    # If robot_lib exposes a robot object, prefer it
    rb = getattr(robot_lib, 'robot', robot_lib)

    speak('Moving forward')
    try_call(rb, ['forward', 'move_forward', 'drive_forward'], 0.2)
    time.sleep(0.5)

    speak('Turning left')
    try_call(rb, ['turn_left', 'left', 'rotate_left'], 0.25)
    time.sleep(0.5)

    speak('Stopping')
    try_call(rb, ['stop', 'halt', 'brake'])

    print('Moves OK')
except Exception as e:
    print('Moves failed:', e)
    print('robot_lib API:', public_names(robot_lib))

## Shutdown

In [ ]:
try:
    if rclpy.ok():
        rclpy.shutdown()
except Exception:
    pass

print('Done')